In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

In [2]:
X_train = pd.read_csv("gisette_train.data",sep=' ',header=None)
y_train = pd.read_csv("gisette_train.labels",sep=' ',header=None)
X_train.dropna(axis=1,how='all',inplace=True)
X_test = pd.read_csv("gisette_valid.data",sep=' ',header=None)
y_test = pd.read_csv("gisette_valid.labels",sep=' ',header=None)
X_test.dropna(axis=1,how='all',inplace=True)

In [3]:
ss=StandardScaler()
ss.fit(X_train)
tf_X_train=ss.transform(X_train)
tf_X_test=ss.transform(X_test)

In [4]:
tf_X_train = tf_X_train.T
tf_X_test=tf_X_test.T
y_train=y_train.replace(-1,0)
y_test=y_test.replace(-1,0)


In [6]:
eta_list = [0.1, 0.01, 0.001]
lamb = 0.0001
m=6000
n_iteration = 300
cost=[]
missclassif_trainlist,missclassif_testlist=[],[]
for eta in eta_list:
  theta = np.zeros((X_train.shape[1],1))
  cost_dummy=[]
  for i in range(n_iteration):
    z = np.dot(theta.T, tf_X_train)
    A = 1/(1+(np.exp(-z)))
    cost_dummy.append((-1/m) * np.sum(y_train.values.T * np.log(A) + (1 - y_train.values.T) * np.log(1 - A), axis=1) + (lamb/m) * np.sum(theta**2, axis=0))
    theta = theta - (eta * np.dot(tf_X_train, (A.T - y_train.values))/m) - (eta * (lamb/m) * theta)
  cost.append(cost_dummy)
  Z_train = np.dot(theta.T,tf_X_train)
  h_x_train = 1/(1+np.exp(-Z_train))
  y_pred_train = np.where(h_x_train<0.5,0,1)
  Z_test = np.dot(theta.T,tf_X_test)
  h_x_test = 1/(1+np.exp(-Z_test))
  y_pred_test = np.where(h_x_test<0.5,0,1)
  missclassif_trainlist.append((1-accuracy_score(y_train,y_pred_train.T))*6000)
  missclassif_testlist.append((1-accuracy_score(y_test, y_pred_test.T))*1000)
print(missclassif_trainlist, missclassif_testlist)


[0.0, 55.99999999999983, 223.00000000000009] [21.000000000000018, 20.000000000000018, 39.000000000000036]


In [ ]:
temp1 = np.array(missclassif_trainlist)
temp1 = temp1/6000
training_error = temp1*100
temp2 = np.array(missclassif_testlist)
temp2 = temp2/1000
test_error = temp2*100

In [ ]:
training_error

In [ ]:
test_error

In [ ]:
fig=plt.figure()
ax=plt.gca()
ax.plot(list(range(n_iteration)), cost[0],label='eta=0.1')
ax.plot(list(range(n_iteration)), cost[1],label='eta=0.01')
ax.plot(list(range(n_iteration)), cost[2],label='eta=0.001')
plt.title('Negative log-likelihood vs The number of iterations')
plt.xlabel('Number of iterations')
plt.ylabel("Negative log-likelihood")
ax.legend()